In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from detoxify import Detoxify

In [ ]:
fine_tuned_model = GPT2LMHeadModel.from_pretrained('fine_tuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('fine_tuned_model')
toxic_model = Detoxify("original")

In [ ]:
def generate_text(prompt_text):
    
    ## Do not generate text if input bio fails toxicity test
    toxicity_rubric_input = toxic_model.predict(prompt_text)
    if toxicity_rubric_input['severe_toxicity'] > .1 or toxicity_rubric_input['threat'] > .01:
        return "Your generated match cannot be shown due to harmful material in your bio. Please modify and try again."
    
    input_ids = tokenizer.encode(prompt_text, return_tensors="pt")
    output = fine_tuned_model.generate(input_ids, min_length=200, max_length=500, num_return_sequences=1, temperature=.9)
    decoded_str = tokenizer.decode(output[0], skip_special_tokens=True)
    
    toxicity_rubric_generated = toxic_model.predict(decoded_str)
    
    ## Do not return generated text if it fails toxicity test
    if toxicity_rubric_generated['severe_toxicity'] > .1 or toxicity_rubric_generated['threat'] > .01:
        return "Your generated match cannot be shown. Please try again."
    
    ## Return generated bio if generated bio passes toxicity tests
    return decoded_str
